# 比較 LDA topic numbers

參考資料
- https://medium.com/pyladies-taiwan/%E4%BB%A5-jieba-%E8%88%87-gensim-%E6%8E%A2%E7%B4%A2%E6%96%87%E6%9C%AC%E4%B8%BB%E9%A1%8C-%E4%BA%94%E6%9C%88%E5%A4%A9%E4%BA%BA%E7%94%9F%E7%84%A1%E9%99%90%E5%85%AC%E5%8F%B8%E6%AD%8C%E8%A9%9E%E5%88%86%E6%9E%90-ii-fdf5d3708662


In [1]:
import os
import numpy as np
import pandas as pd
from collections import Counter
from enum import Enum
from pprint import pprint
import matplotlib.pyplot as plt

import nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models

import pyLDAvis
import pyLDAvis.gensim
from gensim.models.ldamodel import LdaModel

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
                    level=logging.DEBUG)

2019-11-03 15:23:08,113 : DEBUG : Loaded backend module://ipykernel.pylab.backend_inline version unknown.


## 自定義 data types and functions

In [2]:
class ContentType(Enum):
    TIT = 'title'
    ABS = 'abstract'
    AUT = 'author'
    SEC = 'section'
    
def get_contents(content_type):
    all_contents = []
    dataset_path = '../dataset'
    for file in os.listdir(dataset_path):
        file_path = os.path.join(dataset_path, file)
        if os.path.isfile(file_path):
            with open(file_path) as f:  
                line = f.readlines()
                if content_type == ContentType.AUT:
                    line = line[1]
                elif content_type == ContentType.SEC:
                    line = line[2]
                elif content_type == ContentType.ABS:
                    line = line[3]
                else:
                    line = line[0]
                line = line.strip()
                all_contents.append(line)
        else:
            print(file_path + ' does not exist.')
    return all_contents


def get_all_titles():
    return get_contents(ContentType.TIT)

def get_all_authors():        
    return get_contents(ContentType.AUT)

def get_all_sections():
    return get_contents(ContentType.SEC)

def get_all_abstracts():
    return get_contents(ContentType.ABS)

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            token = wordnet_lemmatizer.lemmatize(token, pos='v')
            token = wordnet_lemmatizer.lemmatize(token, pos='n')
            result.append(token)
    return result

def get_chart_data(num_topics,num_words,topics):
    buff = 300
    K = num_topics
    topicWordProbMat = topics
    
    #columns = ['1','2','3','4','5']
    columns = range(1,num_topics+1)

    df = pd.DataFrame(columns = columns)
    pd.set_option('display.width', 1000)

    # 40 will be resized later to match number of words in DC
    zz = np.zeros(shape=(buff,K))

    last_number = 0
    DC = {}

    for x in range (num_words): #取每個topic前10個字
        data= pd.DataFrame(columns=columns,index=[0])
        for i in range(num_topics):
            data[columns[i]] = ""
        df = df.append(data,ignore_index=True)  

    for line in topicWordProbMat:
        topic_id,words = line #一個line是一個topic
        probs = words.split("+")
        y = 0 #用來算第幾個word
        for pr in probs:    
            a = pr.split("*")
            df.iloc[y,topic_id] = a[1] #該word

            if a[1] in DC:
                zz[DC[a[1]]][topic_id] = a[0] #該word的機率
            else:
                zz[last_number][topic_id] = a[0]
                DC[a[1]] = last_number
                last_number = last_number+1
            y = y + 1

    return (df,DC,zz)

def show_words_table(df):  
    print(df)
    print('\n')
    
def show_dictionary(DC):
    print(DC)
    print('字典字數：',len(DC))
    print('\n')

def show_probs_table(zz):
    print(zz)
    print(zz.shape)
    
def show_heapmap(DC,zz):
    %matplotlib inline

    zz = np.resize(zz,(len(DC.keys()),zz.shape[1]))

    for val, key in enumerate(DC.keys()):
            plt.text(-3.5, val + 0.1, key,
                     horizontalalignment='right',
                     verticalalignment='center'
                     )

    #plt.figure(figsize=(10,50))
    plt.imshow(zz, cmap='hot', interpolation='nearest',aspect=0.5)#'auto'
    plt.show()

### 取出所有摘要

In [3]:
contents = get_all_abstracts()
print('共',len(contents),'篇論文\n')

documents = pd.DataFrame(data=contents,columns=['abstract'])
documents['index'] = documents.index
documents[:10]

共 1343 篇論文



,abstract,index
0,We consider the problem of actively eliciting ...,0
1,We investigate the task of distractor generati...,1
2,The most common representation formalisms for ...,2
3,Statistical relational learning models are pow...,3
4,Multimodal representation learning is gaining ...,4
5,Reinforcement learning (RL) has shown its adva...,5
6,Selecting appropriate tutoring help actions th...,6
7,Recognizing time expressions is a fundamental ...,7
8,"When facing large-scale image datasets, online...",8
9,Temporal modeling in videos is a fundamental y...,9


### 預處理的全部論文摘要

In [4]:
processed_docs = documents['abstract'].map(preprocess)
processed_docs[:10]

0    [consider, problem, actively, elicit, preferen...
1    [investigate, task, distractor, generation, mu...
2    [common, representation, formalism, plan, desc...
3    [statistical, relational, learn, model, powerf...
4    [multimodal, representation, learn, gain, deep...
5    [reinforcement, learn, show, advantage, image,...
6    [select, appropriate, tutor, help, action, acc...
7    [recognize, time, expression, fundamental, imp...
8    [face, large, scale, image, datasets, online, ...
9    [temporal, model, video, fundamental, challeng...
Name: abstract, dtype: object

## Dataset

### 產生字典

In [5]:
dictionary = gensim.corpora.Dictionary(processed_docs)
print('共',len(dictionary),'個字\n')

2019-11-03 15:23:14,173 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-11-03 15:23:14,289 : INFO : built Dictionary(6927 unique tokens: ['active', 'actively', 'adaptive', 'aggregation', 'algorithm']...) from 1343 documents (total 121739 corpus positions)


共 6927 個字



### 產生 bag of words corpus

In [6]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
print('共',len(bow_corpus),'筆')

共 1343 筆


### 產生 TF-IDF Corpus

In [7]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
print('共',len(corpus_tfidf),'筆')

2019-11-03 15:23:15,909 : INFO : collecting document frequencies
2019-11-03 15:23:15,910 : INFO : PROGRESS: processing document #0
2019-11-03 15:23:15,939 : INFO : calculating IDF weights for 1343 documents and 6926 features (84803 matrix non-zeros)


共 1343 筆


## Train LDA models

In [8]:
num_topics = 10
num_words = 10

### Running LDA using TF-IDF

In [9]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
                                       num_topics=num_topics, 
                                       id2word=dictionary, 
                                       passes=30,
                                       iterations=50,
                                       eval_every=1)

2019-11-03 15:23:18,754 : INFO : using symmetric alpha at 0.1
2019-11-03 15:23:18,755 : INFO : using symmetric eta at 0.1
2019-11-03 15:23:18,760 : INFO : using serial LDA version on this node
2019-11-03 15:23:18,773 : INFO : running online LDA training, 10 topics, 30 passes over the supplied corpus of 1343 documents, updating every 6000 documents, evaluating every ~1343 documents, iterating 50x with a convergence threshold of 0.001000
2019-11-03 15:23:18,774 : INFO : training LDA model using 3 processes
2019-11-03 15:23:18,798 : DEBUG : worker process entering E-step loop
2019-11-03 15:23:18,821 : DEBUG : worker process entering E-step loop
2019-11-03 15:23:18,824 : DEBUG : getting a new job
2019-11-03 15:23:18,831 : DEBUG : getting a new job
2019-11-03 15:23:18,831 : DEBUG : worker process entering E-step loop
2019-11-03 15:23:18,852 : DEBUG : getting a new job
2019-11-03 15:23:19,275 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #1343/1343, outstanding queue size 

2019-11-03 15:23:25,446 : INFO : topic diff=0.101941, rho=0.462671
2019-11-03 15:23:25,475 : DEBUG : bound: at document #0
2019-11-03 15:23:26,145 : INFO : -11.339 per-word bound, 2591.1 perplexity estimate based on a held-out corpus of 1343 documents with 7958 words
2019-11-03 15:23:26,662 : INFO : PROGRESS: pass 4, dispatched chunk #0 = documents up to #1343/1343, outstanding queue size 1
2019-11-03 15:23:27,004 : DEBUG : processing chunk #0 of 1343 documents
2019-11-03 15:23:27,008 : DEBUG : performing inference on a chunk of 1343 documents
2019-11-03 15:23:27,375 : DEBUG : 1324/1343 documents converged within 50 iterations
2019-11-03 15:23:27,389 : DEBUG : processed chunk, queuing the result
2019-11-03 15:23:27,392 : DEBUG : result put
2019-11-03 15:23:27,395 : DEBUG : updating topics
2019-11-03 15:23:27,406 : INFO : topic #6 (0.100): 0.003*"label" + 0.002*"class" + 0.002*"learn" + 0.002*"text" + 0.002*"image" + 0.002*"model" + 0.002*"multi" + 0.002*"agent" + 0.002*"method" + 0.002

2019-11-03 15:23:33,702 : DEBUG : processing chunk #0 of 1343 documents
2019-11-03 15:23:33,704 : DEBUG : performing inference on a chunk of 1343 documents
2019-11-03 15:23:33,927 : DEBUG : 1340/1343 documents converged within 50 iterations
2019-11-03 15:23:33,941 : DEBUG : processed chunk, queuing the result
2019-11-03 15:23:33,943 : DEBUG : result put
2019-11-03 15:23:33,945 : DEBUG : updating topics
2019-11-03 15:23:33,945 : DEBUG : getting a new job
2019-11-03 15:23:33,955 : INFO : topic #8 (0.100): 0.001*"symbolic" + 0.001*"permutation" + 0.001*"generator" + 0.001*"edit" + 0.001*"program" + 0.001*"dimensionality" + 0.001*"session" + 0.001*"distributional" + 0.001*"regret" + 0.001*"discussion"
2019-11-03 15:23:33,956 : INFO : topic #7 (0.100): 0.002*"community" + 0.001*"nod" + 0.001*"loop" + 0.001*"worker" + 0.001*"event" + 0.001*"brain" + 0.001*"hyperbolic" + 0.001*"program" + 0.001*"puzzle" + 0.001*"riemannian"
2019-11-03 15:23:33,958 : INFO : topic #9 (0.100): 0.004*"learn" + 0.

2019-11-03 15:23:39,825 : DEBUG : processed chunk, queuing the result
2019-11-03 15:23:39,828 : DEBUG : result put
2019-11-03 15:23:39,831 : DEBUG : updating topics
2019-11-03 15:23:39,831 : DEBUG : getting a new job
2019-11-03 15:23:39,841 : INFO : topic #4 (0.100): 0.001*"minority" + 0.001*"english" + 0.001*"parse" + 0.001*"sense" + 0.001*"distribute" + 0.001*"bilingual" + 0.001*"circulant" + 0.001*"equation" + 0.001*"topology" + 0.001*"blame"
2019-11-03 15:23:39,842 : INFO : topic #0 (0.100): 0.003*"translation" + 0.003*"hash" + 0.003*"sentence" + 0.002*"token" + 0.002*"dependency" + 0.002*"english" + 0.002*"proposal" + 0.002*"convolution" + 0.002*"mcmc" + 0.001*"paraphrase"
2019-11-03 15:23:39,843 : INFO : topic #8 (0.100): 0.001*"permutation" + 0.001*"symbolic" + 0.001*"edit" + 0.001*"session" + 0.001*"distributional" + 0.001*"regret" + 0.001*"discussion" + 0.001*"distortion" + 0.001*"grasp" + 0.001*"interview"
2019-11-03 15:23:39,844 : INFO : topic #5 (0.100): 0.002*"traffic" + 0

2019-11-03 15:23:46,001 : DEBUG : getting a new job
2019-11-03 15:23:46,011 : INFO : topic #5 (0.100): 0.002*"review" + 0.001*"traffic" + 0.001*"actor" + 0.001*"constant" + 0.001*"shoot" + 0.001*"verb" + 0.001*"idiom" + 0.001*"coco" + 0.001*"teach" + 0.001*"turnover"
2019-11-03 15:23:46,012 : INFO : topic #7 (0.100): 0.001*"worker" + 0.001*"loop" + 0.001*"brain" + 0.001*"hyperbolic" + 0.001*"riemannian" + 0.001*"centrality" + 0.001*"internship" + 0.001*"blameworthiness" + 0.001*"destination" + 0.001*"cooperation"
2019-11-03 15:23:46,013 : INFO : topic #3 (0.100): 0.002*"gnns" + 0.002*"buyer" + 0.001*"seller" + 0.001*"estimator" + 0.001*"monotone" + 0.001*"reader" + 0.001*"deviation" + 0.001*"revenue" + 0.001*"news" + 0.001*"exchange"
2019-11-03 15:23:46,015 : INFO : topic #1 (0.100): 0.002*"color" + 0.001*"sparsity" + 0.001*"event" + 0.001*"toll" + 0.001*"super" + 0.001*"life" + 0.001*"treatment" + 0.001*"mesh" + 0.001*"sketch" + 0.001*"pixel"
2019-11-03 15:23:46,017 : INFO : topic dif

2019-11-03 15:23:51,926 : INFO : topic #7 (0.100): 0.001*"worker" + 0.001*"loop" + 0.001*"brain" + 0.001*"hyperbolic" + 0.001*"riemannian" + 0.001*"stackelberg" + 0.001*"centrality" + 0.001*"internship" + 0.001*"blameworthiness" + 0.001*"destination"
2019-11-03 15:23:51,927 : INFO : topic #0 (0.100): 0.004*"hash" + 0.002*"token" + 0.002*"proposal" + 0.002*"convolution" + 0.002*"mcmc" + 0.002*"english" + 0.002*"activation" + 0.002*"person" + 0.002*"dictionary" + 0.002*"mutual"
2019-11-03 15:23:51,929 : INFO : topic #4 (0.100): 0.002*"minority" + 0.001*"english" + 0.001*"parse" + 0.001*"bilingual" + 0.001*"circulant" + 0.001*"equation" + 0.001*"german" + 0.001*"distribute" + 0.001*"topology" + 0.001*"blame"
2019-11-03 15:23:51,930 : INFO : topic diff=0.023236, rho=0.214811
2019-11-03 15:23:51,952 : DEBUG : bound: at document #0
2019-11-03 15:23:52,473 : INFO : -10.213 per-word bound, 1186.7 perplexity estimate based on a held-out corpus of 1343 documents with 7958 words
2019-11-03 15:23:

2019-11-03 15:23:57,723 : INFO : topic #4 (0.100): 0.002*"minority" + 0.001*"english" + 0.001*"parse" + 0.001*"topology" + 0.001*"bilingual" + 0.001*"circulant" + 0.001*"equation" + 0.001*"german" + 0.001*"distribute" + 0.001*"blame"
2019-11-03 15:23:57,724 : INFO : topic #8 (0.100): 0.001*"permutation" + 0.001*"distributional" + 0.001*"edit" + 0.001*"session" + 0.001*"regret" + 0.001*"discussion" + 0.001*"symbolic" + 0.001*"distortion" + 0.001*"grasp" + 0.001*"interview"
2019-11-03 15:23:57,725 : INFO : topic diff=0.012209, rho=0.197367
2019-11-03 15:23:57,747 : DEBUG : bound: at document #0
2019-11-03 15:23:58,255 : INFO : -10.199 per-word bound, 1175.4 perplexity estimate based on a held-out corpus of 1343 documents with 7958 words
2019-11-03 15:23:58,629 : INFO : PROGRESS: pass 25, dispatched chunk #0 = documents up to #1343/1343, outstanding queue size 1
2019-11-03 15:23:58,931 : DEBUG : processing chunk #0 of 1343 documents
2019-11-03 15:23:58,933 : DEBUG : performing inference o

2019-11-03 15:24:03,453 : INFO : topic #6 (0.100): 0.002*"right" + 0.001*"subgoals" + 0.001*"patch" + 0.001*"gaze" + 0.001*"activation" + 0.001*"person" + 0.001*"workflow" + 0.001*"metrical" + 0.001*"judgment" + 0.001*"clone"
2019-11-03 15:24:03,454 : INFO : topic diff=0.007013, rho=0.183582
2019-11-03 15:24:03,476 : DEBUG : bound: at document #0
2019-11-03 15:24:03,979 : INFO : -10.192 per-word bound, 1170.0 perplexity estimate based on a held-out corpus of 1343 documents with 7958 words
2019-11-03 15:24:04,350 : INFO : PROGRESS: pass 29, dispatched chunk #0 = documents up to #1343/1343, outstanding queue size 1
2019-11-03 15:24:04,665 : DEBUG : processing chunk #0 of 1343 documents
2019-11-03 15:24:04,667 : DEBUG : performing inference on a chunk of 1343 documents
2019-11-03 15:24:04,834 : DEBUG : 1343/1343 documents converged within 50 iterations
2019-11-03 15:24:04,843 : DEBUG : processed chunk, queuing the result
2019-11-03 15:24:04,846 : DEBUG : result put
2019-11-03 15:24:04,850

In [10]:
vis_data = pyLDAvis.gensim.prepare(lda_model_tfidf, corpus_tfidf, dictionary)
# 在 notebook 中显示可视化结果，需要调用 display 方法，或者执行 “pyLDAvis.enable_notebook()” ，即可在 notebook 中自动展示可视化结果，无需再调用 display
pyLDAvis.display(vis_data)

2019-11-03 15:24:19,289 : DEBUG : performing inference on a chunk of 1343 documents
2019-11-03 15:24:19,790 : DEBUG : 1343/1343 documents converged within 50 iterations
/Users/alexis/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [ ]:
file_name = '../models/lda_unigram_tfidf_topic_'+ str(num_topics) +'.model'
lda_model_tfidf.save(file_name)